# train 데이터셋의 context 데이터 탐색
1. 데이터 기본 정보 확인
2. 데이터 중복 확인 및 데이터 탐색
3. 언어의 종류 확인
4. 데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)

# 0. 데이터셋 및 탐색 준비

In [1]:
# 데이터셋 부르기 위한 datasets 라이브러리 다운로드
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00


In [105]:
# 탐색에 필요한 라이브러리 import
from datasets import load_from_disk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter
import re

In [3]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [4]:
# 데이터셋 불러오기
path = "your_path"
dataset = load_from_disk(path)
print(dataset)
print(dataset['train'])

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})


In [5]:
train_df = dataset_to_df(dataset["train"])
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


# 1. 데이터 기본 정보 확인

In [6]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [7]:
# 데이터의 개수
print(f"데이터 개수 : {len(train_df)}")

데이터 개수 : 3952


In [ ]:
# 랜덤 샘플링한 데이터
# 전체 데이터 중 200개를 랜덤으로 뽑기
sample_context = random.sample(list(train_df["context"]), 200)

print(sample_context)

In [ ]:
# 랜덤 샘플된 데이터 직접 확인하기
for context in sample_context:
  print(context)
  print("=" * 50)

- 확인된 전처리 필요한 문자 <br>
  - `\\n`, `\n`, `\n*`, `\\n\\n`, `\n\n`

- 확인된 언어의 종류<br>
  1. 한국어
  2. 영어
  3. 일본어
  4. 프랑스어
  5. 한자
  6. 아랍어

- 제거가 필요한지 애매한 문자<br>
  - `가우스 그룹#s-1.3`

- 특이한 특수 문자
  - `△사용자`

# 1. 데이터 중복 확인 및 데이터 탐색


In [10]:
# 특정 칼럼을 기준으로 중복된 값을 Counter 함수로 찾은 뒤 리턴하는 함수
def count_duplication(column):
  count = dict(Counter(train_df[column]))
  count = sorted(count.items(),
                 reverse=True,
                 key=lambda item:item[1])

  return count

In [11]:
def print_duplication(column):
  total = len(train_df)
  unique_column = len(train_df[column].unique())

  print(f"전체 데이터 개수 : {total}")
  print(f"unique한 {column} 개수 : {unique_column}")

  if total != unique_column:
    print(f"중복된 값이 있습니다. 중복된 값 개수 : {total - unique_column}")
  else:
    print("중복된 값이 없습니다.")

## document_id 기준으로 중복 확인


In [12]:
# document_id 중복 여부
print_duplication("document_id")

전체 데이터 개수 : 3952
unique한 document_id 개수 : 3340
중복된 값이 있습니다. 중복된 값 개수 : 612


In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [14]:
train_df[train_df["document_id"] == document_count[0][0]]

,title,context,question,answer_start,answer_text,document_id
201,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",평상시 기독교선교사들로부터 차별적 대우를 받은 윤치호가 생각한 미국의 중대한 속성은?,645,인종주의,5284
1426,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호가 미국의 특징이 인종주의라고 생각하게 만든 인물은?,570,기독교선교사들,5284
1622,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",조선에서도 윤치호를 유색 인종이라는 이유로 차별한 사람은?,254,미국인 선교사,5284
2199,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호는 누구의 설교를 듣고 충격을 받았나?,824,남부 출신 남감리교회 목사들,5284


In [15]:
train_df[train_df["document_id"] == document_count[3][0]]

,title,context,question,answer_start,answer_text,document_id
110,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴의 두 번째 임기는 어떤 스캔들로 인해 내리막을 걷게 되었나?,257,화이트워터 사건,4633
3034,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴은 1996년 재선에서 누구를 꺾고 선거를 이겼나요?,188,자신의 공화당 상대 후보 밥 돌,4633
3818,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,두 번째 선거에서 빌 클린턴은 누구를 상대로 승리하였나?,202,밥 돌,4633


## title 기준으로 중복 확인

In [16]:
# title 중복 여부 확인
print_duplication("title")

전체 데이터 개수 : 3952
unique한 title 개수 : 2716
중복된 값이 있습니다. 중복된 값 개수 : 1236


In [ ]:
# title 중복 개수 확인
title_count = count_duplication("title")
print(title_count)

In [18]:
train_df[train_df["title"] == title_count[0][0]]

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
171,윤치호,"1903년 1월 함경도 안핵사로 임명되어 함경남도 함흥에 파견되었고, 7월 천안군수...",윤치호가 중앙 정계에서 다시 활동하기 전에 일어난 전투는?,616,러일전쟁,5295
201,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",평상시 기독교선교사들로부터 차별적 대우를 받은 윤치호가 생각한 미국의 중대한 속성은?,645,인종주의,5284
294,윤치호,서재필이 배재학당의 젊은 학생들과 애국적인 시민을 독립협회로 모으는 데 기여했다면 ...,서재필은 어디에서 공부하는 학생들을 독립협회로 모이게 했나요?,5,배재학당,5342
415,윤치호,1895년 귀국 직후부터 그는 영어 실력을 바탕으로 그는 조선을 방문한 선교사들의 ...,윤치호가 아버지 윤웅렬을 설득해 교회와 학교를 지을 땅을 제공했던 선교사의 이름은?,77,조세핀 애턴 캠밸,5289
583,윤치호,"1935년 조만식, 김성수의 주도로 고희연이 성대히 개최되었다. 이광수 등이 참석했...",이정무의 아들이 이화여전의 이사에 앉은 것은 몇년도인가?,708,1936년,5321
985,윤치호,"1935년 조만식, 김성수의 주도로 고희연이 성대히 개최되었다. 이광수 등이 참석했...",일본인들에게 송도고보는 일본의 어느 학교보다 더 크다고 소문났나?,849,와세다 대학,5321
996,윤치호,"3·1 운동\n1919년 3월 3·1 운동 당시, 독립운동가들로부터 국민대표로 서명...",윤치호가 일부 독립운동 지도자들로부터 비난 받았던 일기의 내용은?,1122,“나는 국경일에 일장기를 게양하는 것을 반대하지 않는다. 왜냐하면 우리가 일본의 통...,5307
1050,윤치호,그러나 다시 1891년 미국 조지아 주 옥스퍼드에 정착한 뒤 다시 에모리 대학(Em...,윤치호는 서재필을 찾아간 후 어느 지역으로 갔나?,317,상하이,5282
1073,윤치호,그는 곧 밴더빌트 대학교 신학부에 입학하였다. 밴더빌트 대학교 재학 중에 그는 감옥...,윤치호가 조지아주에서 재입학한 학교는?,1206,에모리 대학(Oxford College of Emory University),5281


In [19]:
train_df[(train_df["title"] == title_count[0][0]) & (train_df["answer_text"] == "다산 정약용")]

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1212,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",천주교로 종교를 바꿨다는 의심을 받았던 학자는?,300,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


## document_id가 같으면서 answer_start가 같은지 확인

In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [32]:
# document_id가 같으면서 answer_start가 같은 데이터 추출, document_id 값 추출
document_list = []
for count in document_count:
  if count[1] > 1:
    document_count_df = train_df[(train_df["document_id"] == count[0])]
    same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
    if len(same_answer_start_df) > 0:
      document_list.append(count[0])

print(document_list)

[5322, 8428, 7623, 6653, 8190, 6716, 7139, 7833, 25068, 49393, 15663, 33391, 14917, 53065, 5031, 57931, 36795, 5282, 35777, 6711, 51137, 6907, 33784, 47393, 47169, 6329, 32475, 6541, 20027, 10686, 6773]


In [25]:
document_count_df = train_df[(train_df["document_id"] == 5322)]
same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
same_answer_start_df

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


In [37]:
document_answer_df = pd.DataFrame()
for document in document_list:
  document_count_df = train_df[(train_df["document_id"] == document)]
  same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
  document_answer_df = pd.concat([document_answer_df, same_answer_start_df], ignore_index=True)

document_answer_df.head(10)

,title,context,question,answer_start,answer_text,document_id
0,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322
2,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램과 대비되는 프로그램은?,328,명령형 프로그램,8428
3,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램의 알고리즘에 대한 정의와 반대인 개념은?,328,명령형 프로그램,8428
4,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀목 가운데 가장 큰 과와 생김새가 비슷한 과는?,562,현삼과,7623
5,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀과와 유사하게 생긴 식물은?,562,현삼과,7623
6,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",분과에 한 개의 씨가 들어있지 않은 풀의 종류는?,562,현삼,7623
7,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이는 무엇으로 이루어져있나?,445,화산암,6653
8,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이를 이루는 돌의 종류는 무엇인가?,445,화산암,6653
9,베트남의 역사,베트남 전쟁\n\n 베트남 전쟁은 1964년 통킹만 사건을 구실로 미국이 북 베트남...,제2차 인도차이나 전쟁이 끝났던 해는?,131,1975년,8190


# 3. 언어의 종류 확인

In [38]:
language_train_df = train_df
language_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [39]:
# 언어 분류 함수
def classify_language(text):
    has_english = False
    has_korean = False
    has_japanese = False
    has_chinese = False
    has_arabic = False
    has_other = False

    for char in text:
        # 영어 (A-Z, a-z)
        if 'A' <= char <= 'Z' or 'a' <= char <= 'z':
            has_english = True
        # 한글 (유니코드 범위: U+AC00 ~ U+D7A3)
        elif '가' <= char <= '힣':
            has_korean = True
        # 일본어 히라가나 (U+3040 ~ U+309F) 및 가타카나 (U+30A0 ~ U+30FF)
        elif '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
            has_japanese = True
        # 한자 (유니코드 범위: U+4E00 ~ U+9FFF)
        elif '\u4E00' <= char <= '\u9FFF':
            has_chinese = True
        # 아랍어 (유니코드 범위: U+0600 ~ U+06FF)
        elif '\u0600' <= char <= '\u06FF':
            has_arabic = True
        # 기타 문자
        else:
            has_other = True

    # 결과 반환
    languages = []
    if has_korean:
        languages.append("Korean")
    if has_english:
        languages.append("English")
    if has_japanese:
        languages.append("Japanese")
    if has_chinese:
        languages.append("Chinese")
    if has_arabic:
        languages.append("Arabic")
    if has_other and not languages:
        languages.append("Other")

    return ", ".join(languages) if languages else "Other"

# 'context' 열에서 언어 분석 후 새로운 열 추가
language_train_df['languages'] = language_train_df['context'].apply(classify_language)

# 결과 출력
print(language_train_df[['context', 'languages']])

                                                context  \
0     미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1     '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2     강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3     불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4     동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   
...                                                 ...   
3947  이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...   
3948  애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...   
3949  1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...   
3950  원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...   
3951  2008년 2월 28일 실시된 2008년 함부르크 주의회 선거에서 기민련은 과반수 ...   

                     languages  
0              Korean, English  
1              Korean, English  
2     Korean, English, Chinese  
3     Korean, English, Chinese  
4     Korean, English, Chinese  
...                        ...  
3947  Korean, English, Chinese  
3948           Korean, Engl

In [41]:
language_train_df.head(10)

,title,context,question,answer_start,answer_text,document_id,languages
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293,"Korean, English"
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638,"Korean, English"
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028,"Korean, English, Chinese"
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146,"Korean, English, Chinese"
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334,"Korean, English, Chinese"
5,용아병,델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소...,카드모스의 부하들이 간 곳에는 무엇이 있었는가?,91,드래곤,37482,"Korean, English"
6,호반,"제27회와 제73회에 출현한다. 관우는 자신을 저지하는 공수, 맹탄, 한복, 변희를...",관우를 불태워 죽이려한 사람 누구인가?,68,형양태수 왕식,24395,"Korean, English, Chinese"
7,제50노섬브리안 보병사단,이집트로 이어지는 해안 도로는 1개의 사단만이 방어할 수 있었으며 엘 아뎀 요새와 ...,참호 속에 무기와 장비를 버리고 도주한 집단은?,583,이탈리아군,43399,"Korean, English"
8,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,제2차 세계 대전에 참전하여 사망한 자식은?,195,큰아들 유,10426,"Korean, English, Chinese"
9,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?,861,왕대마을,15766,"Korean, English, Chinese"


In [55]:
# context 데이터에 사용된 언어 종류 확인
unique_languages = set()

for language in list(language_train_df["languages"].unique()):
    languages = language.split(', ')
    unique_languages.update(languages)

result = list(unique_languages)

print(result)

['English', 'Korean', 'Chinese', 'Arabic', 'Japanese']


In [73]:
contains_df = language_train_df[language_train_df["context"].str.contains("폴 르 노르망")]
print(contains_df.to_string())

                      title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# 4.데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)


In [113]:
# 개행 문자 카운트 함수 (by 정휘님)
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        all_lines.append([i, new_line_count])
    return all_lines

In [114]:
newline_counts = np.array(get_newline(train_df["context"]))
print(np.sum(newline_counts[:,1],axis=0))

22808


In [110]:
# 특수 문자를 찾는 함수 정의
def find_special_characters(text, pattern):
    # 정규 표현식을 사용하여 특수 문자 찾기
    return re.findall(pattern, text)

# context에서 모든 특수 문자를 찾고 중복 제거
all_special_chars = set()
#pattern = r'[^a-zA-Z0-9가-힣\s]' # 알파벳, 숫자, 한글, 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9fff]' # 알파벳, 숫자, 한글, 한자 공백 제외
pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u0600-\u06FF]' # 알파벳, 숫자, 한글, 한자, 일본어, 아랍어 공백 제외
for context in train_df['context']:
    special_chars = find_special_characters(context, pattern)
    all_special_chars.update(special_chars)

# 결과 출력
print(f"찾은 특수 문자 종류: {all_special_chars}")
print(f"특수 문자의 개수: {len(all_special_chars)}")

찾은 특수 문자 종류: {'ć', 'φ', '"', 'ί', 'ˈ', '…', '.', 'ე', '>', 'স', 'ᐨ', 'м', 'µ', '”', '←', '〔', 'ù', '↓', 'à', '¾', 'ि', 'ó', 'ㅁ', 'і', '⑪', 'Μ', 'ს', 'ַ', '【', '＞', '～', 'ש', 'р', 'Н', '¹', 'đ', 'Ｍ', 'æ', '②', 'ח', 'Ⅱ', 'ल', 'क', 'Ó', 'ώ', '―', '!', '∧', '。', 'ा', '<', '‘', 'έ', 'ে', '▲', '+', 'Р', 'ნ', 'ვ', 'П', 'й', '=', 'מ', 'ς', 'ா', 'स', 'ׇ', '\uf000', '–', '㎜', 'ত', '」', 'λ', 'Σ', 'Þ', 'ê', 'ர', '１', 'ო', '≫', ')', 'χ', '☆', 'М', '□', '⑨', 'ό', 'Đ', 'ஜ', 'β', '·', 'დ', 'त', '}', '⑦', 'ㅇ', 'ㅚ', '×', 'Л', '≪', 'ச', '´', '±', '「', '‧', 'ī', 'ờ', 'י', 'ᛚ', '़', 'ι', '８', '^', 'α', '`', '•', 'ğ', '＝', 'γ', '㈜', '’', 'В', 'и', 'ი', '΄', 'а', 'ु', 'η', 'ł', 'ᚢ', 'া', 'Φ', 'κ', '%', '⇌', 'ν', '“', 'с', '\xad', 'ë', 'ὔ', 'ā', '○', 'ी', '３', 'ά', 'ơ', 'œ', '）', 'თ', '․', '③', '—', 'ழ', '∼', ',', '》', 'ː', '↑', '、', 'θ', '２', 'π', 'Α', 'ᚴ', 'ύ', 'ʲ', 'ோ', '$', 'გ', 'Π', 'е', 'ш', '｢', ']', 'ִ', 'ü', '（', 'х', '㎠', 'ы', 'ა', '】', 'ㅕ', 'л', 'ạ', 'ὁ', '：', 'ხ', 'Ｖ', 'т', '&', 'ɔ', '④', '『', 'ǫ'